In [20]:
# remove this cell if there is no GPU
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # 
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [21]:
##################### Parasite Segmentation: binary mask generation #################
import tensorflow as tf
import segmentation_models as sm
import os
import cv2
import numpy as np
from tqdm import tqdm
sm.set_framework('tf.keras')

model = sm.Unet('vgg19', encoder_weights=None)
model.load_weights('E:/malaria_data/vgg19_color_ntl_dataset12_shuffle_nocrops_augmented.hdf5')

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-4),
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.iou_score],
)


test_path = 'E:/malaria_data/dataset_2/test/img'
predict_path = 'E:/malaria_data/dataset_2/test/predict'

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                   samplewise_center=True,
                                                                   samplewise_std_normalization=True)  # preprocessing_function=preprocess_input

file_list = os.listdir(test_path)

for file_name in tqdm(file_list):
    
    #original image reading
    full_path = os.path.join(test_path, file_name)
    img= cv2.imread(full_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_rgb = cv2.resize(img_rgb, (224, 224), interpolation = cv2.INTER_LINEAR) 
    x = np.expand_dims(img_rgb, axis=0)
    
    test_iterator = test_datagen.flow(x,batch_size=1)
    x = test_iterator.next()
    result = model.predict(x, verbose=0)
    result = np.squeeze(result, axis=0)
    result = np.where(result > 0.5, 255, 0)
    #save predicted mask
    full_path = os.path.join(predict_path, file_name)
    cv2.imwrite(full_path, result)

100%|████████████████████████████████████████████████████████████████████████████████| 124/124 [00:11<00:00, 10.92it/s]


In [22]:
############# Binary mask resize to original image size ###################
import cv2
import numpy as np
import os 
from tqdm import tqdm


def resize_function(input_directory, output_directory, new_w, new_h):
    
    original_file_list = os.listdir(input_directory)
    
    for img_file in tqdm(original_file_list):
        
        # read gt image in binary format
        full_path = os.path.join(input_directory, img_file)
        mask= cv2.imread(full_path, 0) # 0 for grayscale flag
        (thresh, mask) = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
        
        #resize
        dsize = (new_w,new_h)
        # resize image
        mask = cv2.resize(mask, dsize)
        (thresh, mask) = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
        
        # copy and paste original image
        full_path = os.path.join(output_directory, img_file)
        cv2.imwrite(full_path, mask)


input_directory = "E:/malaria_data/dataset_2/test/predict"
output_directory= "E:/malaria_data/dataset_2/test/predict"

#dataset1
#new_w = 2592
#new_h = 1944

#dataset2
new_w = 1382
new_h = 1030

resize_function(input_directory, output_directory, new_w, new_h)

100%|████████████████████████████████████████████████████████████████████████████████| 124/124 [00:01<00:00, 85.05it/s]


In [23]:
##### Parasite contours delineation on original image, number of parasites is saved within the name of the image ########### 
import cv2 
import numpy as np
from cv2 import boxPoints
import os
from tqdm import tqdm


def box_fun(gray_img,color_img):
    (cnts, _) = cv2.findContours(gray_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    c = sorted(cnts, key=cv2.contourArea, reverse=True)
    gray_rgb = cv2.cvtColor(gray_img, cv2.COLOR_BGR2RGB)
    #print("{}{}".format('Detected parasites: ',len(c)))
    for i in range(0,len(c)):
        #use the 2 statement to trace boxes and replace c[i] by box
        #rect = cv2.minAreaRect(c[i])
        #box =  np.int0(boxPoints(rect))
        cv2.drawContours(color_img, [c[i]], -1, (0, 0, 255), 3) 
        
    return len(c), color_img 


# images and mask directories
img_directory = 'E:/malaria_data/dataset_2/test/img'
mask_directory = 'E:/malaria_data/dataset_2/test/predict'

# contours output directory
original_bb_directory = 'E:/malaria_data/dataset_2/test/contours'

#gt_file_list = os.listdir(gt_directory)
original_file_list = os.listdir(img_directory)

for original_file_name in tqdm(original_file_list):
    
    #mask images reading and binary thresholding
    mask_path = os.path.join(mask_directory, original_file_name)
    mask_img= cv2.imread(mask_path, 0) # 0 for grayscale flag
    (thresh, mask_img) = cv2.threshold(mask_img, 127, 255, cv2.THRESH_BINARY)
    
    #original image reading
    original_path = os.path.join(img_directory, original_file_name)
    original_img= cv2.imread(original_path)
    
    #parasite counting for each image and contour tracing
    parasite_count, original_img_bound = box_fun(mask_img,original_img)
    
    #save original img with boxes
    new_name = original_file_name.split(".png", 1)
    new_name = os.path.join(original_bb_directory, new_name[0]+"_"+str(parasite_count)+"_parasites.png")
    cv2.imwrite(new_name, original_img_bound)
    
    

100%|████████████████████████████████████████████████████████████████████████████████| 124/124 [00:17<00:00,  7.17it/s]


In [24]:
 ###### Crop the largest parasite from each image and save it. Requires orginal image and associated binary mask ##########
import cv2 
import os
import numpy as np
from cv2 import boxPoints

def savecrop(original_img, original_file_name, output_directory, x,y,w,h, window_h, window_w):
    
    extension=".png"
    max_h= window_h # should be 224
    max_w= window_w # should be 224
    delta_h=max_h-h
    delta_w=max_w-w
    
    #position center
    a=y-(delta_h // 2)
    b=y-(delta_h // 2)+max_h
    e=x-(delta_w // 2)
    f=x-(delta_w // 2)+max_w    
    if a>=0 and b<=original_img.shape[0] and e>=0 and f<=original_img.shape[1]:
        
        new_img2=original_img[a:b,e:f]#img
        #writes the new file in the Crops folder
        new_name = original_file_name.split(extension, 1)[0]
        full_path_file_name = os.path.join(output_directory, new_name +".png")
        cv2.imwrite(full_path_file_name, new_img2) 
        return
        
    
        
    #position 4 middle left
    a=y-(delta_h // 2)
    b=a+max_h
    e=x
    f=e+max_w     
    if a>=0 and b<=original_img.shape[0] and f<=original_img.shape[1]:
        
        new_img2=original_img[a:b,e:f]#img
        #writes the new file in the Crops folder
        new_name = original_file_name.split(extension, 1)[0]
        full_path_file_name = os.path.join(output_directory, new_name +".png")
        cv2.imwrite(full_path_file_name, new_img2)
        return
    
    #position 6 middle right
    a=y-(delta_h // 2)
    b=a+max_h
    e=x-delta_w
    f=e+max_w    
    if a>=0 and b<=original_img.shape[0] and e>=0 and f<=original_img.shape[1]:
        
        new_img2=original_img[a:b,e:f]#img
        #writes the new file in the Crops folder
        new_name = original_file_name.split(extension, 1)[0]
        full_path_file_name = os.path.join(output_directory, new_name +".png")
        cv2.imwrite(full_path_file_name, new_img2)
        return

    
    #position 2 up center
    a=y
    b=y+max_h
    e=x-(delta_w // 2)
    f=x-(delta_w // 2)+max_w      
    if e>=0 and b<=original_img.shape[0] and f<=original_img.shape[1]:
        
        new_img2=original_img[a:b,e:f]#img
        #writes the new file in the Crops folder
        new_name = original_file_name.split(extension, 1)[0]
        full_path_file_name = os.path.join(output_directory, new_name +".png")
        cv2.imwrite(full_path_file_name, new_img2)
        return
    
    #position 1 up left
    a=y
    b=y+max_h
    e=x
    f=x+max_w        
    if b<=original_img.shape[0] and f<=original_img.shape[1]:
        
        new_img2=original_img[a:b,e:f]#img
        #writes the new file in the Crops folder
        new_name = original_file_name.split(extension, 1)[0]
        full_path_file_name = os.path.join(output_directory, new_name +".png")
        cv2.imwrite(full_path_file_name, new_img2)
        return
    
    #position 3 up right
    a=y
    b=y+max_h
    e=x-delta_w
    f=x-delta_w+max_w      
    if b<=original_img.shape[0] and e>=0 and f<=original_img.shape[1]:
        
        new_img2=original_img[a:b,e:f]#img
        #writes the new file in the Crops folder
        new_name = original_file_name.split(extension, 1)[0]
        full_path_file_name = os.path.join(output_directory, new_name +".png")
        cv2.imwrite(full_path_file_name, new_img2)
        return
    
    #position 8 down center
    a=y-delta_h
    b=a+max_h
    e=x-(delta_w // 2)
    f=e+max_w   
    if a>=0 and b<=original_img.shape[0] and e>=0 and f<=original_img.shape[1]:
        
        new_img2=original_img[a:b,e:f]#img
        #writes the new file in the Crops folder
        new_name = original_file_name.split(extension, 1)[0]
        full_path_file_name = os.path.join(output_directory, new_name +".png")
        cv2.imwrite(full_path_file_name, new_img2)
        return
    
    #position 7 down left
    a=y-delta_h
    b=a+max_h
    e=x
    f=e+max_w   
    if a>=0 and b<=original_img.shape[0] and f<=original_img.shape[1]:
        
        new_img2=original_img[a:b,e:f]#img
        #writes the new file in the Crops folder
        new_name = original_file_name.split(extension, 1)[0]
        full_path_file_name = os.path.join(output_directory, new_name +".png")
        cv2.imwrite(full_path_file_name, new_img2)
        return   
            
    #position 9 down right
    a=y-delta_h
    b=a+max_h
    e=x-delta_w
    f=e+max_w  
    if a>=0 and b<=original_img.shape[0] and e>=0 and f<=original_img.shape[1]:
        
        new_img2=original_img[a:b,e:f]#img
        #writes the new file in the Crops folder
        new_name = original_file_name.split(extension, 1)[0]
        full_path_file_name = os.path.join(output_directory, new_name +".png")
        cv2.imwrite(full_path_file_name, new_img2)


def crop_largest(gray_img,color_img, original_file_name, output_directory):
    
    (cnts, _) = cv2.findContours(gray_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    c = sorted(cnts, key=cv2.contourArea, reverse=True)
    
    gray_rgb = cv2.cvtColor(gray_img, cv2.COLOR_BGR2RGB)
    #print("{}{}".format('Detected parasites: ',len(c)))
    if (len(c)>0):
        max_area = 0
        index_max=-1
        flag = 0
        for i in range(0,len(c)):
            x, y, w, h = cv2.boundingRect(c[i])
            if((w*h > max_area)): #50176 and (w*h <= 50176)
                index_max = i
                max_area = w*h
                flag = 1
        
        if(flag == 1):
            x,y,w,h = cv2.boundingRect(c[index_max])
            savecrop(color_img, original_file_name, output_directory, x,y,w,h, 224, 224)
    else:
        fn.append(original_file_name)
        
        

mask_directory = "E:/malaria_data/dataset_2/test/predict"
original_directory = "E:/malaria_data/dataset_2/test/img"
output = "E:/malaria_data/dataset_2/test/crops"

extension = ".png"
#gt_file_list = os.listdir(gt_directory)
original_file_list = os.listdir(original_directory)
fn=[]
for original_file_name in tqdm(original_file_list):
    
    #mask images reading and binary thresholding
    prefix_name = original_file_name.split(extension, 1)
    mask_file_name = prefix_name[0] +extension
    
    mask_path = os.path.join(mask_directory, mask_file_name)
    mask_img= cv2.imread(mask_path, 0) # 0 for grayscale flag
    #gt_img=cv2.cvtColor(gt_img, cv2.COLOR_BGR2GRAY)
    (thresh, mask_img) = cv2.threshold(mask_img, 127, 255, cv2.THRESH_BINARY)
    
    #original image reading
    original_path = os.path.join(original_directory, original_file_name)
    original_img= cv2.imread(original_path)
    
    #crop largest parasite if it exists
    crop_largest(mask_img,original_img,original_file_name, output)
    
print("List of images without parasites: %s" % fn)    

        

100%|████████████████████████████████████████████████████████████████████████████████| 124/124 [00:09<00:00, 13.72it/s]

List of images without parasites: ['Trip 038 Day 1 01-12-05 Image 11 add_9.png', 'Trip 053 Day 2 19-11-05 Image 3_12.png']


In [25]:
############ Load classifier and assign to each crop the predicted species ###############
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
from tqdm import tqdm


model = load_model('E:/malaria_data/LightNet_classification_mixed.hdf5')

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


# The right species of all crops from this test is Falciparum
test_path = 'E:/malaria_data/dataset_2/test/crops'


test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                   samplewise_center=True,
                                                                   samplewise_std_normalization=True)  # preprocessing_function=preprocess_input

file_list = os.listdir(test_path)
plasmodium = ['Falciparum', 'Malariae', 'Ovale', 'Vivax']
for file_name in file_list: #tqdm
    
    #original image reading
    #print(file_name)
    full_path = os.path.join(test_path, file_name)
    img= cv2.imread(full_path)
    img_rgb = cv2.resize(img, (224, 224), interpolation = cv2.INTER_LINEAR) 
    img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
    x = np.expand_dims(img_rgb, axis=0)
    
    test_iterator = test_datagen.flow(x,batch_size=1)
    x = test_iterator.next()
    result = model.predict(x, verbose=0)
    
    index = np.argmax(result,-1) # get class index of max value
    print('%s [%s]' % (file_name,plasmodium[index[0]]) ) 
    
    

Trip 017 Day 1 19-10-05 Image 14 add_4.png [Falciparum]
Trip 017 Day 1 19-10-05 Image 14 add_7.png [Falciparum]
Trip 017 Day 1 19-10-05 Image 15 add_1.png [Falciparum]
Trip 017 Day 1 19-10-05 Image 15 add_6.png [Falciparum]
Trip 017 Day 1 19-10-05 Image 16 add_10.png [Falciparum]
Trip 017 Day 1 19-10-05 Image 16 add_2.png [Falciparum]
Trip 017 Day 1 19-10-05 Image 17 add_11.png [Falciparum]
Trip 017 Day 1 19-10-05 Image 17 add_14.png [Falciparum]
Trip 022 Day 1 08-11-05 Image 10 add_14.png [Falciparum]
Trip 022 Day 1 08-11-05 Image 10 add_6.png [Falciparum]
Trip 022 Day 1 08-11-05 Image 11 add_11.png [Falciparum]
Trip 022 Day 1 08-11-05 Image 11 add_16.png [Falciparum]
Trip 022 Day 1 08-11-05 Image 9 add_12.png [Falciparum]
Trip 022 Day 1 08-11-05 Image 9 add_8.png [Falciparum]
Trip 029 Day 1 22-11-05 Image 10 add_4.png [Falciparum]
Trip 029 Day 1 22-11-05 Image 10 add_8.png [Falciparum]
Trip 029 Day 1 22-11-05 Image 11 add_13.png [Falciparum]
Trip 029 Day 1 22-11-05 Image 11 add_5.png